In [ ]:
N=100 #length of time series
lagged=theano.shared(df['lagged'].values) #previous value in time series
vals=theano.shared(df['vals'].values)
#The data is hierarchical, with 4 levels. The alpha and beta coefficients for the 
#auto-regressive function inherit from distributions on the first of 4 levels, which
#inherits from the next highest level, etc up to global alpha and beta distributions.
#These are multivariate Gaussian Random Walks. Because the time series is unstationary,
#these serve as time varying auto-regressive coefficients, i.e. each time period n has a
#global coefficient for alpha and beta.
with pm.Model() as model:
    #Highest level of hierarchy:
    #global beta and alpha mus are inherited by the Random Walk alphas and betas
    global_beta_mu=pm.Normal('global_beta_mu',mu=0,sd=100,shape=(N-1,var1_count))
    #var1_count is 5 (the number of unique categories in my first variable)
    global_alpha_mu=pm.Normal('global_alpha_mu',mu=0,sd=100,shape=(N-1,var1_count))
    #The Cholesky matrix models the covariance between time series
    packed_L_β = pm.LKJCholeskyCov('packed_L_beta', n=var1_count, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    L_β = pm.expand_packed_triangular(var1_count, packed_L_β)
    packed_L_α = pm.LKJCholeskyCov('packed_L_alpha', n=var1_count, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    L_α = pm.expand_packed_triangular(var1_count, packed_L_α)
    alpha = pm.MvGaussianRandomWalk('alpha', mu=global_alpha_mu[:,var1_indexes], shape=(N, var1_count), chol=L_α)
    beta = pm.MvGaussianRandomWalk('beta', mu=global_beta_mu[:,var1_indexes], shape=(N, var1_count), chol=L_β)
    #Second level of hierarhcy:
    #vars12_count is 10 (the unique combinations of categories from first 2 variables)
    vars12_alpha_sd=pm.HalfCauchy('vars12_alpha_sd',beta=4,shape=(N,vars12_count))
    vars12_beta_sd=pm.HalfCauchy('vars12_beta_sd',beta=4,shape=(N,vars12_count))
    #The offset allows the sampler to fully explore the likely values, preventing it from 
    #getting stuck
    vars12_alpha_offset=pm.Normal('vars12_alpha_offset',mu=0,sd=1,shape=(N,vars12_count)) 
    vars12_beta_offset=pm.Normal('vars12_beta_offset',mu=0,sd=1,shape=(N,vars12_count))
    #vars12_alpha_m and vars12_beta_m is the mu for the second hierarchical level - they 
    #draw from the Random Walk distributions alpha and beta defined above, with the offset 
    #and SD to help the sampler
    vars12_alpha_m=pm.Deterministic('vars12_alpha_m',alpha[:,vars12_indexes] + 
                        vars12_alpha_offset * vars12_alpha_sd[:,var12_indexes])
    vars12_beta_m=pm.Deterministic('vars12_beta_m',beta[:,vars12_indexes] + 
                        vars12_beta_offset*vars12_beta_sd[:,var12_indexes])
    #Third level of hierarchy:
    #vars123_count is 130 (unique combinations of first three variables)
    vars123_alpha_sd = pm.HalfCauchy('vars123_alpha_sd', beta=4, shape=(N,vars123_count)) 
    vars123_beta_sd = pm.HalfCauchy('vars123_beta_sd', beta=4, shape=(N,vars123_count))
    vars123_alpha_offset=pm.Normal('vars123_alpha_offset',mu=0,sd=1,shape=(N,vars123_count)) 
    vars123_beta_offset=pm.Normal('vars123_beta_offset',mu=0,sd=1,shape=(N,vars123_count))
    #The third level draws from the distributions with means vars12_alpha_m and vars12_beta_m
    vars123_alpha_m=pm.Deterministic('vars123_alpha_m',vars12_alpha_m[:,vars123_indexes] +
                        vars123_alpha_offset*vars123_alpha_sd[:,vars123_indexes])
    vars123_beta_m=pm.Deterministic('vars123_beta_m',vars12_beta_m[:,vars123_indexes]+
                        vars123_beta_offset*vars123_beta_sd[:,vars123_indexes])    
    #Fourth level of hierarchy:
    #vars1234_count is 520 (unique combinations of all four variables)
    vars1234_alpha_sd = pm.HalfCauchy('vars1234_alpha_sd', beta=4, shape=(N,vars1234_count))
    vars1234_beta_sd = pm.HalfCauchy('vars1234_beta_sd', beta=4, shape=(N,vars1234_count))
    vars1234_alpha_offset=pm.Normal('vars1234_alpha_offset',mu=0,sd=1,shape=(N,vars1234_count))
    vars1234_beta_offset=pm.Normal('vars1234_beta_offset',mu=0,sd=1,shape=(N,vars1234_count))
    vars1234_alpha_m=pm.Deterministic('vars1234_alpha_m',vars123_alpha_m[:,vars1234_indexes]+
                        var123_alpha_offset*vars1234_alpha_sd[:,vars1234_indexes])
    vars1234_beta_m=pm.Deterministic('vars1234_beta_m',vars123_beta_m[:,vars1234_indexes] +
                        occ_month_yr_beta_offset*vars1234_beta_sd[:,vars1234_indexes])  
    #The indexing makes the auto-regressive function hard to follow, but this is simply
    #an equation with form y_t = alpha + beta * y_t-1
    regression=pm.Deterministic('regression',
                vars1234_alpha_m[df['dates'].values,df['vars1234_index'].values]+
                vars1234_beta_m[df['vals'].values,
                df['vars1234_index'].values]*lagged)
    #Uniform prior for the error
    obs_error=pm.Uniform('obs_error',lower=0,upper=100)
    #The likelihood distribution, with mu equal to the auto-regressive function above
    likelihood=pm.Normal('likelihood',mu=regression,sd=obs_error,observed=vals)    
    #Find the parameters through sampling 1000 times
    trace=pm.sample(1000)